In [3]:
import pandas as pd

from mainnet_launch.database.postgres_operations import (
    _exec_sql_and_cache,
    get_full_table_as_df_with_tx_hash,
)

from mainnet_launch.database.views import fetch_rich_autopool_destinations_table
from mainnet_launch.database.schema.full import *
from mainnet_launch.slack_messages.post_message import post_slack_message, post_message_with_table, SlackChannel
from mainnet_launch.constants import *

from datetime import datetime, timedelta



In [34]:
hour_ago = datetime.utcnow() - timedelta(days=1)
week_ago = datetime.utcnow() - timedelta(days=7)

def _fetch_tokens_sent_to_liqudation_row() -> pd.DataFrame:
    query = f"""
        SELECT
            incentive_token_balance_updated.liquidation_row, 
            incentive_token_balance_updated.chain_id,
            ARRAY_AGG(DISTINCT incentive_token_balance_updated.token_address) AS token_addresses
        FROM 
            incentive_token_balance_updated
        JOIN
            transactions 
            ON incentive_token_balance_updated.tx_hash = transactions.tx_hash 
            AND incentive_token_balance_updated.chain_id = transactions.chain_id
        JOIN
            blocks ON transactions.block = blocks.block AND transactions.chain_id = blocks.chain_id

        WHERE 
            blocks.datetime >= '{week_ago.isoformat()}'

        GROUP BY 
            liquidation_row,
            incentive_token_balance_updated.chain_id
    """
    df = _exec_sql_and_cache(query)
    return df.explode('token_addresses', ignore_index=True)



def _fetch_incentive_tokens_actually_sold():
    query = f"""
        SELECT
            incentive_token_swapped.liquidation_row, 
            incentive_token_swapped.chain_id,
            ARRAY_AGG(DISTINCT incentive_token_swapped.sell_token_address) AS token_addresses
        FROM 
            incentive_token_swapped
        JOIN
            transactions 
            ON incentive_token_swapped.tx_hash = transactions.tx_hash 
            AND incentive_token_swapped.chain_id = transactions.chain_id
        JOIN
            blocks ON transactions.block = blocks.block AND transactions.chain_id = blocks.chain_id

        WHERE 
            blocks.datetime >= '{hour_ago.isoformat()}'

        GROUP BY 
            liquidation_row,
            incentive_token_swapped.chain_id
    """
    df = _exec_sql_and_cache(query)
    return df.explode('token_addresses', ignore_index=True)


actually_sold = _fetch_incentive_tokens_actually_sold()
expected_tokens_to_be_sold = _fetch_tokens_sent_to_liqudation_row()
print("Expected to be sold:")
print(expected_tokens_to_be_sold.shape)
print("Actually sold:")
print(actually_sold.shape)

chain_name = {chain.chain_id: chain.name for chain in ALL_CHAINS}
expected_tokens_to_be_sold['chain_name'] = expected_tokens_to_be_sold['chain_id'].map(chain_name)
actually_sold['chain_name'] = actually_sold['chain_id'].map(chain_name)
# actually_sold = actually_sold[actually_sold['chain_id'] != 1].reset_index(drop=True)
diff = expected_tokens_to_be_sold.merge(actually_sold, how="outer", indicator=True)
expected_but_not_actually_sold = diff[diff["_merge"] == "left_only"].drop(columns="_merge")
expected_but_not_actually_sold

expected_but_not_actually_sold

Expected to be sold:
(24, 3)
Actually sold:
(25, 3)


,liquidation_row,chain_id,token_addresses,chain_name


In [39]:
actually_sold.groupby(['liquidation_row', 'chain_name']).size().to_clipboard()

In [ ]:
# liquidation_row	chain_name	0
# 0x610Ffeb00B8312B0540DED300c683227CB3E3AB5	arb	5
# 0x7571dE594A92379c0A053E2A5004514057c10B5D	base	8
# 0xD3132ce50e7471cC6130Ac5Aa553149dc3B2A018	plasma	1
# 0xF570EA70106B8e109222297f9a90dA477658d481	eth	9
# 0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e	sonic	2


SyntaxError: invalid syntax (1422240662.py, line 1)

In [52]:
from multicall import Call
from mainnet_launch.data_fetching.get_state_by_block import safe_normalize_with_bool_success, get_raw_state_by_blocks, build_blocks_to_use, get_state_by_one_block
from mainnet_launch.constants import BASE_CHAIN
import plotly.express as px

blocks = build_blocks_to_use(BASE_CHAIN)


call = Call(
            '0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1',
            ["balanceOf(address)(uint256)", '0x7571dE594A92379c0A053E2A5004514057c10B5D'],
            [('base liquidation row BAL', safe_normalize_with_bool_success),],
        )


bal_balance_df = get_raw_state_by_blocks(
    calls = [call],
    blocks=blocks,
    chain=BASE_CHAIN, 
)

bal_balance_df

,base liquidation row BAL
timestamp,
2024-10-18 17:12:33+00:00,0.000000
2024-10-22 19:33:03+00:00,0.000000
2024-11-02 23:59:59+00:00,0.000000
2024-11-03 23:59:59+00:00,0.000000
2024-11-04 23:59:59+00:00,0.000000
...,...
2025-10-24 23:59:59+00:00,1244.752696
2025-10-25 23:59:59+00:00,1380.566448
2025-10-26 23:59:59+00:00,1521.670775


In [55]:
import plotly.express as px

px.bar(bal_balance_df,title= 'BAL in base liquidation row over time')

In [ ]:
# todo check swapped df and see if the thing is there, eg have we really sold bal

In [ ]:
swapped_df = get_full_table_as_df_with_tx_hash(IncentiveTokenSwapped)



,tx_hash,log_index,chain_id,liquidation_row,sell_token_address,buy_token_address,sell_amount,buy_amount,buy_amount_received,block
datetime,,,,,,,,,,
2025-10-29 18:58:17+00:00,0xdb52b7292248cc53f1349ac6d2ac22eca9f92176bf6d...,522,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,123.641534,13.602384,13.698178,37487475
2025-10-29 18:58:17+00:00,0xdb52b7292248cc53f1349ac6d2ac22eca9f92176bf6d...,443,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0xBAa5CC21fd487B8Fcc2F632f3F4E8D37262a0842,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,95.334980,186.674070,187.786353,37487475
2025-10-29 18:58:17+00:00,0xdb52b7292248cc53f1349ac6d2ac22eca9f92176bf6d...,481,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0xA88594D404727625A9437C3f886C7643872296AE,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,216.097930,3.772945,3.798593,37487475
2025-10-29 18:58:17+00:00,0xdb52b7292248cc53f1349ac6d2ac22eca9f92176bf6d...,499,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0x1C7a460413dD4e964f96D8dFC56E7223cE88CD85,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,420.891215,95.556390,96.233202,37487475
2025-10-29 18:58:17+00:00,0xdb52b7292248cc53f1349ac6d2ac22eca9f92176bf6d...,419,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0x61E030A56D33e8260FdD81f03B162A79Fe3449Cd,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,254.386100,1171.998180,1179.650660,37487475
...,...,...,...,...,...,...,...,...,...,...
2024-09-19 19:06:47+00:00,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,239,1,0xBf58810BB1946429830C1f12205331608c470ff5,0xba100000625a3754423978a60c9317c58a424e3D,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,97.656862,0.073584,0.073584,20786622
2024-09-19 19:06:47+00:00,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,212,1,0xBf58810BB1946429830C1f12205331608c470ff5,0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,405.885565,0.065661,0.065661,20786622
2024-09-19 19:06:47+00:00,0xeaff1c935812795d7c751554c06a6f725d0523fe985d...,283,1,0xBf58810BB1946429830C1f12205331608c470ff5,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,6.322527,0.005347,0.005347,20786622


In [58]:
swapped_df[swapped_df['sell_token_address'] == '0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1'][['sell_amount', 'chain_id', 'sell_token_address']]

,sell_amount,chain_id,sell_token_address
datetime,,,
2025-10-29 18:55:39+00:00,20.691057,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
2025-10-29 06:55:33+00:00,20.689141,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
2025-10-28 18:55:37+00:00,20.692299,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
2025-10-28 06:55:35+00:00,20.697305,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
2025-10-27 18:55:35+00:00,20.700308,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
...,...,...,...
2024-11-09 00:55:27+00:00,0.107105,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
2024-11-08 12:55:29+00:00,0.107116,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1
2024-11-08 00:55:29+00:00,0.157782,8453,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1


In [ ]:
('0x610Ffeb00B8312B0540DED300c683227CB3E3AB5', 'arb')

In [26]:
expected_tokens_to_be_sold['chain_id'].value_counts()

chain_id
8453     8
42161    5
9745     1
146      1
Name: count, dtype: int64

In [ ]:


expected_tokens_to_be_sold = _fetch_all_incentive_tokens_sent_to_liqudation_rows()
expected_tokens_to_be_sold = expected_tokens_to_be_sold.explode().explode().dropna().reset_index()
at_least_some_sold_tokens = swapped_df.groupby(["liquidation_row", "chain_id"])["sell_token_address"].apply(list)
at_least_some_sold_tokens = at_least_some_sold_tokens.explode().dropna().reset_index()

# at_least_some_sold_tokens.rename(columns={"sell_token_address": "at_least_some_sold_tokens"}, inplace=True)
# expected_tokens_to_be_sold.rename(columns={"sell_token_address": "expected_tokens_to_be_sold"}, inplace=True)
expected_tokens_to_be_sold_tuples = list(expected_tokens_to_be_sold.itertuples(index=False, name=None))
at_least_some_sold_tokens_tuples = list(at_least_some_sold_tokens.itertuples(index=False, name=None))

for expected in expected_tokens_to_be_sold_tuples:
    if expected not in at_least_some_sold_tokens_tuples:
        print(f"Liquidation row {expected[0]} on chain {expected[1]} expected to sell token {expected[2]} but did not.")


,liquidation_row,chain_id,sell_token_address
0,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8
1,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x1509706a6c66CA549ff0cB464de88231DDBe213B
2,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x7dfF72693f6A4149b17e7C6314655f6A9F7c8B33
3,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x7dfF72693f6A4149b17e7C6314655f6A9F7c8B33
4,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8
...,...,...,...
318,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x5Af79133999f7908953E94b7A5CF367740Ebee35
319,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x5Af79133999f7908953E94b7A5CF367740Ebee35
320,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x2D0E0814E62D80056181F5cd932274405966e4f0
321,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x5Af79133999f7908953E94b7A5CF367740Ebee35


: 

,liquidation_row,chain_id,at_least_some_sold_tokens
0,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x912CE59144191C1204E64559FE8253a0e49E6548
1,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x40BD670A58238e6E230c430BBb5cE6ec0d40df48
2,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8
3,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x7dfF72693f6A4149b17e7C6314655f6A9F7c8B33
4,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,42161,0x1509706a6c66CA549ff0cB464de88231DDBe213B
...,...,...,...
59,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x5Af79133999f7908953E94b7A5CF367740Ebee35
60,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x5Af79133999f7908953E94b7A5CF367740Ebee35
61,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x5Af79133999f7908953E94b7A5CF367740Ebee35
62,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,146,0x2D0E0814E62D80056181F5cd932274405966e4f0


: 

: 

: 

In [ ]:
swapped_df

,tx_hash,log_index,chain_id,liquidation_row,sell_token_address,buy_token_address,sell_amount,buy_amount,buy_amount_received,block
datetime,,,,,,,,,,
2025-10-29 02:54:39+00:00,0x94111cc979d19789192a429fe71479f5d9fdfbfa3931...,2,146,0xf3b137219325466004AEb91CAa0A0Bdd2A8afc8e,0x5Af79133999f7908953E94b7A5CF367740Ebee35,0x29219dd400f2Bf60E5a23d13Be72B486D4038894,7.143427,3.818333,3.818333,52343053
2025-10-28 22:50:01+00:00,0x56844b5714372e648cb292b465e0eaff44a7e9468eed...,14,9745,0xD3132ce50e7471cC6130Ac5Aa553149dc3B2A018,0x6100E367285b01F48D07953803A2d8dCA5D19873,0xB8CE59FC3717ada4C02eaDF9682A9e934F625ebb,1982.501539,721.076880,726.167539,4762395
2025-10-28 21:37:23+00:00,0x539e77b4ea77021c731534175ec6a4845d9a0ca58b25...,149,42161,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,0x912CE59144191C1204E64559FE8253a0e49E6548,0xaf88d065e77c8cC2239327C5EDb3A432268e5831,1159.897056,358.651740,361.188701,394415607
2025-10-28 21:37:23+00:00,0x539e77b4ea77021c731534175ec6a4845d9a0ca58b25...,79,42161,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,0x40BD670A58238e6E230c430BBb5cE6ec0d40df48,0xaf88d065e77c8cC2239327C5EDb3A432268e5831,92.562179,177.746512,177.746512,394415607
2025-10-28 21:37:23+00:00,0x539e77b4ea77021c731534175ec6a4845d9a0ca58b25...,69,42161,0x610Ffeb00B8312B0540DED300c683227CB3E3AB5,0x040d1EdC9569d4Bab2D15287Dc5A4F10F56a56B8,0xaf88d065e77c8cC2239327C5EDb3A432268e5831,1.565410,1.549548,1.560447,394415607
...,...,...,...,...,...,...,...,...,...,...
2025-10-27 18:56:11+00:00,0x883397d321fed3f992fe21513ddb19bf5c1502b2b9ae...,815,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0xA88594D404727625A9437C3f886C7643872296AE,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,16164.211391,308.465520,310.624065,37401012
2025-10-27 18:56:11+00:00,0x883397d321fed3f992fe21513ddb19bf5c1502b2b9ae...,784,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0xBAa5CC21fd487B8Fcc2F632f3F4E8D37262a0842,0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913,121.032201,249.799080,251.543144,37401012
2025-10-27 18:55:35+00:00,0xf74214f9e8c67b11c5f84c4cfd592530601b7b388e38...,642,8453,0x7571dE594A92379c0A053E2A5004514057c10B5D,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1,0x4200000000000000000000000000000000000006,20.700308,0.005146,0.005182,37400994


: 

In [ ]:
# maybe,

# https://www.alchemy.com/docs/data/prices-api/prices-api-endpoints/prices-api-endpoints/get-token-prices-by-address


# flag if there is more than > $100 in any of them?

# flag if a liqudation row has not had at least one liqudation in the last 2 days.

# Either works

: 